In [1]:
import os
import re

import numpy as np
import torch
import tqdm
from utilities import *

### 定义几个辅助函数：
↓👇以下是两个表：单词->单词序号；单词序号->词向量。


word2index：把单词转成索引，索引可以找到词向量

wocabulary_vectors：所有单词的glove embedding，索引是单词id

In [2]:
word_list = []
vocabulary_vectors = []
#大家都用100d
word2index, vocabulary_vectors = get_glove_encoding()

测试一下，看看数据发育正不正常啊

In [3]:
process_sentence(word2index, "train")
process_sentence(word2index, "test")

大哥，你已经预处理过了🤣不过为了保险起见，还是重新预处理一下


100%|██████████| 25000/25000 [00:00<00:00, 3717300.06it/s]


[[174943    152     14 ...     -1     -1     -1]
 [ 26494     23     46 ...     26   2231      4]
 [  6520     74     11 ...     -1     -1     -1]
 [    37     14   2407 ...     -1     -1     -1]
 [    37     14     36 ...     -1     -1     -1]]
[1, 1, 1, 1, 1]
大哥，你已经预处理过了🤣不过为了保险起见，还是重新预处理一下


100%|██████████| 25000/25000 [00:00<00:00, 3118348.90it/s]


[[   41   388     5 ...    -1    -1    -1]
 [ 2018   852   369 ...   179    10     7]
 [   19     7  9897 ...     0 12514   260]
 [   41   822    37 ...    -1    -1    -1]
 [  480 30091    31 ...    -1    -1    -1]]
[1, 1, 1, 1, 1]


👇看一下数据分布&画图图

In [4]:
# import matplotlib.pyplot as plt

# plt.plot(result_train_label[0])
# plt.title("Length distribution of negative instances")
# plt.show()
# plt.plot(result_train_label[1])
# plt.title("Length distribution of positive instances")
# plt.show()
# # result_reallen_train.sort()
# # IQR=result_reallen_train[(int)(len(result_reallen_train)*0.75)]-result_reallen_train[(int)(len(result_reallen_train)*0.25)]
# # fuck=result_reallen_train[(int)(len(result_reallen_train)*0.75)]+1.5*IQR
# # plt.boxplot(result_reallen_train)
# # plt.show()
# # plt.boxplot(result_reallen_test)
# # plt.show()
# words = sorted(word_cnt_train_label[0], key=lambda x: word_cnt_train_label[0][x], reverse=True)[:20]
# plt.bar(words, [word_cnt_train_label[0][w] for w in words])
# plt.title("Words distribution of negative instances")
# plt.show()
# words = sorted(word_cnt_train_label[1], key=lambda x: word_cnt_train_label[1][x], reverse=True)[:20]
# plt.bar(words, [word_cnt_train_label[1][w] for w in words])
# plt.title("Words distribution of positive instances")
# plt.show()


# # print(fuck)


### 叔叔我啊，要开始训练了捏

首先，写一个获取数据的函数

In [5]:
# from model import MyTransformerEncoder
# import dataset

# model = MyTransformerEncoder()